In [3]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import base64
import us as us
from sklearn import preprocessing

In [46]:
# for json formatted info

# with open('data/borough_data.json') as f:
#     borough_data=json.load(f)
# borough_data

with open('data/state_funding.csv') as f:
    funds_df=pd.read_csv(f, delimiter=',')

FileNotFoundError: [Errno 2] No such file or directory: 'data/state_funding.csv'

In [ ]:
funds_df.head()

In [ ]:
# keeping the state and the total funds allocated to each state, for 2021
funds_df = funds_df.drop(labels=['Funding Based on Average Number of Unemployed', 'Funding from Even Distribution', 'CRF Top-Up'], axis=1) #only interested in the total funding
funds_df.loc[:, "Total Funding"] = funds_df.loc[:, "Total Funding"].str.replace(r"$", "", regex=True)
funds_df.loc[:, "Total Funding"] = funds_df.loc[:, "Total Funding"].str.replace(r".", "", regex=True)
funds_df.loc[:, "Total Funding"] = funds_df.loc[:, "Total Funding"].str.replace(r",", "", regex=True) #remove irrelevant characters from numerics
funds_df = pd.melt(funds_df, id_vars=['State'], value_vars=['Total Funding'], ignore_index=True) #reorganizing the dataframe with the states as index values
funds_df = funds_df.drop(labels="variable", axis=1)
funds_df = funds_df.dropna()
funds_df = funds_df.rename({"value":"Total Funds"}, axis=1)
funds_df.head()

In [ ]:
#responsible for converting the monetary values to billions
def convert_to_billions(vals):
    new_vals = []
    for val in vals:
        val = val.replace(val[-8:], "")
        val = val[0:len(val) - 3] + '.' + val[-3:]
        new_vals.append(val)
    return new_vals

vals = funds_df["Total Funds"].values
funds_df["Total Funds"] = pd.Series(convert_to_billions(vals))
funds_df = pd.DataFrame(data=funds_df["Total Funds"].values, index=funds_df["State"], columns=["Total Funds (in billions)"] )
funds_df = funds_df.copy().drop(index=51)

In [ ]:
# saved the df as csv
# funds_df.to_csv('data/cleaned_state_funds.csv')

In [ ]:
with open('data/cleaned_state_funds.csv') as f:
    clean_funds_df=pd.read_csv(f, delimiter=',')

In [ ]:
fig, ax = plt.subplots(figsize=(30,16))
plt.subplots_adjust(hspace=0.5)

ax = sns.barplot(data=clean_funds_df, x=clean_funds_df.State, y=clean_funds_df['Total Funds (in billions)']).set_xticklabels(labels=clean_funds_df.State, rotation=90)

In [ ]:
# clean_funds_df = clean_funds_df.drop(columns="Unnamed: 0")
# clean_funds_df.to_csv('data/cleaned_state_funds.csv', index=False)

In [ ]:
with open('data/CARES_state_funds_2020.csv') as f:
    cares_funds_df=pd.read_csv(f, delimiter=',')

In [ ]:
fig, ax = plt.subplots(figsize=(30,16))
plt.subplots_adjust(hspace=0.5)

ax = sns.barplot(data=cares_funds_df, x=cares_funds_df.State, y=cares_funds_df['Total Funds (in billions)']).set_xticklabels(labels=cares_funds_df.State, rotation=90)

In [ ]:
# cares_funds_df = cares_funds_df.drop(columns="Unnamed: 0")
# cares_funds_df.to_csv('data/CARES_state_funds_2020.csv', index=False)

In [ ]:
# merged 2 datasets into one final funds per state dataset

clean_funds_df.columns = ['State', 'Total Funds for 2021 (in billions)']
cares_funds_df.columns = ['State', 'Total Funds for 2020 (in billions)']
clean_funds_df = clean_funds_df.merge(cares_funds_df, how="inner", on="State")
# clean_funds_df = clean_funds_df.set_index('State')
clean_funds_df.to_csv('data/final_state_funds.csv', index=False)

### Run the cell below to get the new state funds results per year

In [ ]:
with open('data/final_state_funds.csv') as f:
    final_state_funds=pd.read_csv(f, delimiter=',')

In [ ]:
final_state_funds.head()

In [ ]:
# final_state_funds = final_state_funds.set_index('State')

In [ ]:
final_state_funds.dtypes

In [ ]:
# with open('data/2019_Census_US_Population_Data_By_State_Lat_Long.csv') as f:
#     population_df=pd.read_csv(f, delimiter=',')
    
with open('data/final_state_funds_with_pop.csv') as f:
    population_df=pd.read_csv(f, delimiter=',')

In [ ]:
# funds per person
# population_df = population_df.rename(columns={'STATE':'State', 'POPESTIMATE2019':'Estimated Population (2019)', 'lat':'Latitude (of Capital)', 'long':'Longitude (of Capital)'})
# population_df = final_state_funds.merge(population_df, on="State")

# population_df["Funds Per Person (2021)"] = (population_df["Total Funds for 2021 (in billions)"] * (10**9))/population_df["Estimated Population (2019)"]
# population_df["Funds Per Person (2020)"] = (population_df["Total Funds for 2020 (in billions)"] * (10**9))/population_df["Estimated Population (2019)"]
population_df

In [ ]:
# population_df.to_csv('data/final_state_funds_with_pop.csv', index=False)

In [ ]:
# used for population, failed

# pop_cols = ['NAME', 'POPESTIMATE2019']
# population_df = population_df[pop_cols]
# population_df = population_df.groupby(['NAME'], as_index=False).sum()
# population_df = population_df.rename(columns={'NAME':'State'})
# state_pop_funds = final_state_funds.merge(population_df, on="State")

population_df.to_csv('data/final_state_funds_with_pop.csv', index=False)

### Normalizing the values

In [ ]:
with open('data/final_state_funds_with_pop.csv') as f:
    population_df=pd.read_csv(f, delimiter=',')

In [ ]:
# population_df=population_df.loc[population_df['State'] != "District of Columbia"].reset_index().drop(columns='index')

#### Normalized the total funds cols

In [ ]:
funds_2021_arr = np.array(population_df['Total Funds for 2021 (in billions)'])
funds_2020_arr = np.array(population_df['Total Funds for 2020 (in billions)'])
normalized_2021 = preprocessing.normalize([funds_2021_arr])
normalized_2020 = preprocessing.normalize([funds_2020_arr])
normal_list = []
normal_list2 = []
for element in normalized_2021[0]:
    normal_list.append(element)
for element in normalized_2020[0]:
    normal_list2.append(element)

normal_series = pd.Series(normal_list)
normal_series2 = pd.Series(normal_list2)

population_df['Normalized Funds 2021'] = normal_series
population_df['Normalized Funds 2020'] = normal_series2

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))
plt.subplots_adjust(hspace=0.5)
ax=sns.barplot(data=population_df, x='State', y='Normalized Funds 2020').set_xticklabels(labels=population_df.State, rotation=90)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))
plt.subplots_adjust(hspace=0.5)
ax=sns.barplot(data=population_df, x='State', y='Normalized Funds 2021').set_xticklabels(labels=population_df.State, rotation=90)

#### Normalized the per person cols

In [ ]:
funds_per_person_2021_arr = np.array(population_df['Funds Per Person (2021)'])
funds_per_person_2020_arr = np.array(population_df['Funds Per Person (2020)'])
normalized_2021 = preprocessing.normalize([funds_per_person_2021_arr])
normalized_2020 = preprocessing.normalize([funds_per_person_2020_arr])
normalized_list = []
normalized_list2 = []
for element in normalized_2021[0]:
    normalized_list.append(element)
for element in normalized_2020[0]:
    normalized_list2.append(element)

normal_series = pd.Series(normalized_list)
normal_series2 = pd.Series(normalized_list2)

population_df['Normalized Per Person Funds 2021'] = normal_series
population_df['Normalized Per Person Funds 2020'] = normal_series2

In [ ]:
funds_per_person_2021_arr

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))
plt.subplots_adjust(hspace=0.5)
ax=sns.barplot(data=population_df, x='State', y='Normalized Per Person Funds 2020').set_xticklabels(labels=population_df.State, rotation=90)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))
plt.subplots_adjust(hspace=0.5)
ax=sns.barplot(data=population_df, x='State', y='Normalized Per Person Funds 2021').set_xticklabels(labels=population_df.State, rotation=90)

In [ ]:
# population_df.to_csv('data/final_state_funds_with_pop.csv', index=False)

## Processing HRR datasets 

# Don't focus on the HRR datasets, as they are estimates, not official numbers!
- wasted time unfortunately :(

In [ ]:
with open('data/HRR_perc_20.csv') as f:
    HRR_20_df=pd.read_csv(f, delimiter=',')

In [ ]:
# Convert location of hospitals from cities to states

HRR_20_df = HRR_20_df.dropna()
def city_to_state(vals):
    new_vals = []
    for val in vals:
        val = str(us.states.lookup(val[-2:]))
        new_vals.append(val)
    return new_vals

HRR_20_df['HRR'] = city_to_state(HRR_20_df.HRR.values)
HRR_20_df = HRR_20_df.drop(columns=['Adult Population', 'Population 65+'])

In [ ]:
# Dropping the columns containing predicted values of the dataset as we are not trying to predict anything on the hospitals,
# if they were official values from the hospitals, they could be useful.

# HRR_20_df = HRR_20_df.drop(columns=HRR_20_df.columns[7:])
# HRR_20_df.to_csv('data/HRR_info.csv', index=False)


# HRR_info_df.head()
# HRR_20_df.groupby(['HRR']).sum()
# HRR_20_df.dtypes 
# need to replace special characters (% , .) and convert objects to numerics
# for percentage columns, left shift twice

In [ ]:
# with open('data/HRR_info.csv') as f:
#     HRR_info_df=pd.read_csv(f, delimiter=',')

## New Hospital Dataset

### Some Things to Keep mind
- Some columns are unnecessary. Share with others what could possibly be used/what should be kept
- I don't want to drop rows with nulls, NaN errors, because unless duplicated, each row should contain useful info
- Convert the values to int/floats (numerics) 

In [ ]:
with open('data/Patient_Impact_Hospital_Capacity.csv') as f:
    pihc_df=pd.read_csv(f, delimiter=',')
    
pihc_df

In [ ]:
pihc_df.dtypes.to_dict()

In [ ]:
# pihc_df.iloc[:,11:].dtypes == "object"

pihc_df.astype({'previous_week_personnel_covid_vaccinated_doses_administered_7_day_max':'int64'})

# temp_cols = pihc_df.iloc[:,11:].select_dtypes("object").columns
# for row in temp_cols:
#     try:
#         pihc_df.astype({'{row}':'int64'})
#     except:
#         pihc_df.astype({'{row}':'float'})
            
#     pihc_df.iloc[:,11:].select_dtypes("object")
    
# temp_df = pihc_df.iloc[:,11:].select_dtypes("object").astype('int64', errors='ignore')
# temp_df = temp_df.select_dtypes("object").astype('float', errors='ignore')
# temp_df

In [ ]:
temp_df.dtypes.to_dict()

In [ ]:
pihc_df.loc[pihc_df[11:].dtypes == "object"].astype('int64')

In [ ]:
pihc_df.head()

In [ ]:
pihc_df.dtypes.unique()